In [4]:
import os,sys
import time,re
import operator
import numpy as np
import nltk,math

In [5]:
current_dir =  os.getcwd()
corpus_file = current_dir + '/Brown_tagged_train.txt' 
file = open(corpus_file , 'r')
text = file.readlines()

# Calculating the Transition and Emission Probabilities.

In [6]:
text_words = []
text_tags = []

for sentence in text:
    sentence_tags = []
    sentence_words = []
    words = sentence.split(' ')
    for word_tag in words:
        temp = word_tag.rsplit('/',1)
        sentence_tags.append(temp[-1])
        sentence_words.append(temp[0])
    sentence_tags = sentence_tags[:-2]
    sentence_words = sentence_words[:-2]
    sentence_tags.insert(0,'*')
    sentence_tags.insert(0,'*')
    sentence_tags.append('STOP')
    sentence_words.insert(0,'*')
    sentence_words.insert(0,'*')
    sentence_words.append('STOP')
    text_tags.append(sentence_tags)
    text_words.append(sentence_words)
print text_tags
print text_words



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
def get_unigrams(text_tags):
    unigrams = {}
    for sentence in text_tags:
        for token in sentence:
            count = unigrams.get(token,0)
            unigrams[token] = count + 1
    return unigrams
unigrams = get_unigrams(text_tags)
#print unigrams

In [6]:
def get_bigrams(text_tags):
    bigrams = {}
    for sentence in text_tags:
        i = 0
        while i < (len(sentence)-1):
            curr_word = sentence[i]
            next_word = sentence[i+1]
            if curr_word not in bigrams:
                bigrams[curr_word] = {}
            if next_word not in bigrams[curr_word]:
                bigrams[curr_word][next_word] = 1
            else:
                bigrams[curr_word][next_word] += 1
            i += 1
    return bigrams
bigrams = get_bigrams(text_tags)
#print bigrams

In [7]:
def get_trigrams(text_tags):
    trigrams = {}
    for sentence in text_tags:
        i = 0
        while i < (len(sentence)-2):
            curr_word = sentence[i]
            first_next_word = sentence[i+1]
            second_next_word = sentence[i+2]
            if curr_word not in trigrams:
                trigrams[curr_word] = {}
            if first_next_word not in trigrams[curr_word]:
                trigrams[curr_word][first_next_word] = {}
            if second_next_word not in trigrams[curr_word][first_next_word]:
                trigrams[curr_word][first_next_word][second_next_word] = 1
            else:
                trigrams[curr_word][first_next_word][second_next_word] += 1
            i += 1
    return trigrams
trigrams = get_trigrams(text_tags)
#print trigrams


In [8]:
def prob_unigram(unigrams):
    unigram_prob = {}
    total_sum = sum(unigrams.values())
    for word in unigrams:
        unigram_prob[word] = unigrams[word]/float(total_sum)
    return unigram_prob
unigram_prob = prob_unigram(unigrams)
#print unigram_prob

# Transition Probabilities

In [9]:
def prob_bigram(unigrams,bigrams):
    bigram_prob = {}
    for curr_word in bigrams:
        bigram_prob[curr_word] = {}
        for next_word in bigrams[curr_word]:
            bigram_prob[curr_word][next_word] =  bigrams[curr_word][next_word]/float(unigrams[curr_word])
    return bigram_prob
bigram_prob = prob_bigram(unigrams,bigrams)
#print bigram_prob


In [10]:
def prob_trigram(bigrams,trigrams):
    trigram_prob = {}
    for curr_word in trigrams:
        trigram_prob[curr_word] = {}
        for first_next_word in trigrams[curr_word]:
            trigram_prob[curr_word][first_next_word] = {}
            for second_next_word in trigrams[curr_word][first_next_word]:
                trigram_prob[curr_word][first_next_word][second_next_word] = trigrams[curr_word][first_next_word][second_next_word]/float(bigrams[curr_word][first_next_word])
    return trigram_prob
trigram_prob = prob_trigram(bigrams,trigrams)
#print trigram_prob


In [11]:
known_words = set()
unigrams_words = get_unigrams(text_words)
#print unigrams_words

for word in unigrams_words:
    if unigrams_words[word] > 5:
        known_words.add(word)  
#print known_words

In [12]:
text_words_rare = []

for sentence in text_words:
    temp = []
    for word in sentence:
        if word not in known_words:
            word = '__RARE__'
        temp.append(word)
    text_words_rare.append(temp)

# Emission Probabilities

In [13]:
word_given_tag_count = {}
sentence_tagging = zip(text_words,text_tags)

for sent_words,sent_tags in sentence_tagging:
    #POS tagging to each word 
    word_tagging = zip(sent_words,sent_tags)
    for word,tag in word_tagging:
        if tag not in word_given_tag_count:
            word_given_tag_count[tag] = {}
        if word not in word_given_tag_count[tag]:
            word_given_tag_count[tag][word] = 1
        else:
            word_given_tag_count[tag][word] += 1
#print word_given_tag_count

In [14]:
prob_word_given_tag = {}

for tag in word_given_tag_count:
    prob_word_given_tag[tag] = {}
    for word in word_given_tag_count[tag]:
        prob_word_given_tag[tag][word] = word_given_tag_count[tag][word]/float(sum(word_given_tag_count[tag].values()))
#print prob_word_given_tag

# Probability of the observation Sequence

In [15]:

#testinf sequence
seq = 'personal hardship for such employees'

### Forwarding 
def forwading(seq,unigrams,prob_word_given_tag,bigrams):
    seq_words = seq.split(' ')
    seq_len = len(seq_words)
    states = unigrams.keys()
    alpha = [{}]
    seq = seq_words
    if seq_len == 0:
            return []
    for state in states:
        state_prob = 0
        emit_prob = 0
        if state in unigrams:
            state_prob = unigrams[state]/float(sum(unigrams.values()))
        if state in prob_word_given_tag:
            if seq[0] in prob_word_given_tag[state]:
                emit_prob = prob_word_given_tag[state][seq[0]]
        alpha[0][state] = state_prob * emit_prob

    for i in xrange(1,seq_len):
        alpha.append({})
        for state_to in states:
            prob = 0
            emit_prob = 0
            for state_from in states:
                trans_prob = 0
                if state_from in bigrams:
                    if state_to in bigrams[state_from]:
                        trans_prob = bigrams[state_from][state_to]
                prob += alpha[i-1][state_from] * trans_prob
                #print prob
            if state_to in prob_word_given_tag:
                if seq[i] in prob_word_given_tag[state_to]:
                    emit_prob = prob_word_given_tag[state_to][seq[i]]
            alpha[i][state_to] = prob * emit_prob
    return alpha

#print alpha

### backwarding
def backwarding(seq,unigrams,prob_word_given_tag,bigrams):
    seq_words = seq.split(' ')
    seq_len = len(seq_words)
    states = unigrams.keys()
    seq = seq_words
    if seq_len == 0:
            return []
    beta = [{}]
    for state in states:
        beta[0][state] = 1
    for i in xrange(seq_len - 1, 0, -1):
        beta.insert(0,{})
        for state_from in states:
            prob = 0
            for state_to in states:
                trans_prob = 0
                if state_from in bigrams:
                    if state_to in bigrams[state_from]:
                        trans_prob = bigrams[state_from][state_to]
                emit_prob = 0
                if state_to in prob_word_given_tag:
                    if seq[i] in prob_word_given_tag[state_to]:
                        emit_prob = prob_word_given_tag[state_to][seq[i]]
                prob += beta[1][state_to]* trans_prob * emit_prob
            beta[0][state_from] = prob
    return beta
beta = backwarding(seq,unigrams,prob_word_given_tag,bigrams)
#print beta


###evaluate Sentence
def evalute(seq):
    seq_words = seq.split(' ')
    seq_len = len(seq_words)
    prob = 0
    alpha = forwading(seq,unigrams,prob_word_given_tag,bigrams)
    if seq_len == 0:
            return 0
    for state in alpha[seq_len - 1]:
        prob += alpha[seq_len - 1][state]
    return prob

# Identify Tags for Sequence 

In [16]:
def decode(seq,unigrams,prob_word_given_tag,bigrams):
    seq_words = seq.split(' ')
    seq_len = len(seq_words)
    states = unigrams.keys()
    seq = seq_words
    if seq_len == 0:
            return []
    delta = {}
    for state in states:
        state_prob = 0
        emit_prob = 0
        if state in unigrams:
            state_prob = unigrams[state]/float(sum(unigrams.values()))
        if state in prob_word_given_tag:
            if seq[0] in prob_word_given_tag[state]:
                emit_prob = prob_word_given_tag[state][seq[0]]
        delta[state] = state_prob * emit_prob
    
    pre = []
    for i in xrange(1, seq_len):
        delta_bar = {}
        pre_state = {}
        
        for state_to in states:
            max_prob = 0
            max_state = None
            for state_from in states:
                trans_prob = 0
                if state_from in bigrams:
                    if state_to in bigrams[state_from]:
                        trans_prob = bigrams[state_from][state_to]
                prob = delta[state_from] * trans_prob
                if prob > max_prob:
                    max_prob = prob
                    max_state = state_from
            
            emit_prob = 0
            if state_to in prob_word_given_tag:
                if seq[i] in prob_word_given_tag[state_to]:
                    emit_prob = prob_word_given_tag[state_to][seq[i]]
            delta_bar[state_to] = max_prob * emit_prob
            pre_state[state_to] = max_state
        delta = delta_bar
        pre.append(pre_state)
        
    max_state = None
    max_prob = 0
    for state in states:
        if delta[state] > max_prob:
            max_prob = delta[state]
            max_state = state
    
    if max_state is None:
        return []
    
    result = [max_state]
    for i in xrange(seq_len - 1, 0, -1):
        max_state = pre[i - 1][max_state]
        result.insert(0, max_state)
        
    return result

# Test Data

In [20]:
#using untagged brown corpus
for sentence in text_words[0:50]:
    sentence = sentence[2:]
    sentence = sentence[:-1]
    seq = ''
    for word in sentence:
        seq = seq + ' ' + word
    seq =  seq.strip()
    print '\n'
    print "Observation Sequence"
    print seq
    print '\n'
    print 'Probability of the Observation Sequence'
    print evalute(seq)
    print '\n'
    print 'Tags of Observation Sequence'
    tags = decode(seq,unigrams,prob_word_given_tag,bigrams)
    print tags
    print '\n'
    print 'Top 50 words for Sequence Tags'
    
    dis_tags = []
    for tag in tags:
        if tag not in dis_tags:
            dis_tags.append(tag)
            
    for tag in dis_tags:
        x = word_given_tag_count[tag]
        sorted_x = sorted(x.items(), key=operator.itemgetter(1),reverse=True)
        i = 0
        ter = 50
        le = len(sorted_x)
        if le < 50:
            ter = le
        strg = tag + ":  "
        while i < ter:
            strg = strg + sorted_x[i][0].rstrip() + ", "
            i+=1
        print strg
        print '\n'
    print '\n\n\n\n\n\n\n'



Observation Sequence
At that time highway engineers traveled rough and dirty roads to accomplish their duties


Probability of the Observation Sequence
7.96760002813e+12


Tags of Observation Sequence
['ADP', 'DET', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'CONJ', 'ADJ', 'NOUN', 'PRT', 'VERB', 'DET', 'NOUN']


Top 50 words for Sequence Tags
ADP:  of, in, to, for, with, on, that, as, at, by, from, into, In, if, than, about, like, through, over, before, after, If, between, under, because, against, For, so, while, without, until, out, toward, upon, As, At, since, around, during, though, within, across, On, behind, down, per, among, With, whether, Of, 


DET:  the, a, his, The, this, an, which, her, that, their, no, what, my, its, some, any, these, A, This, our, each, your, those, both, another, What, His, every, These, That, No, Her, An, My, whose, Some, Each, Another, Our, whatever, Both, Their, Your, either, Its, Those, Any, Every, Which, neither, 


NOUN:  Af, time, man, way, years, me

ADP:  of, in, to, for, with, on, that, as, at, by, from, into, In, if, than, about, like, through, over, before, after, If, between, under, because, against, For, so, while, without, until, out, toward, upon, As, At, since, around, during, though, within, across, On, behind, down, per, among, With, whether, Of, 


VERB:  was, is, had, be, were, have, are, would, been, said, could, will, can, has, do, made, did, may, must, get, see, should, know, might, go, came, make, went, got, didn't, thought, used, come, found, looked, take, being, don't, knew, going, say, think, felt, took, saw, turned, told, put, asked, seemed, 


DET:  the, a, his, The, this, an, which, her, that, their, no, what, my, its, some, any, these, A, This, our, each, your, those, both, another, What, His, every, These, That, No, Her, An, My, whose, Some, Each, Another, Our, whatever, Both, Their, Your, either, Its, Those, Any, Every, Which, neither, 












Observation Sequence
Table 1 immediately below shows the r

NOUN:  Af, time, man, way, years, men, eyes, people, day, year, Mr., something, hand, one, room, head, door, water, place, state, number, life, face, house, work, home, system, States, night, part, course, hands, nothing, thing, world, side, information, things, end, case, line, anything, point, use, school, Mrs., business, group, voice, moment, 


VERB:  was, is, had, be, were, have, are, would, been, said, could, will, can, has, do, made, did, may, must, get, see, should, know, might, go, came, make, went, got, didn't, thought, used, come, found, looked, take, being, don't, knew, going, say, think, felt, took, saw, turned, told, put, asked, seemed, 


ADV:  not, when, so, only, then, now, more, back, as, where, even, just, too, also, here, there, still, very, again, never, When, away, how, well, about, always, most, much, Then, once, Now, almost, around, later, enough, ever, however, first, rather, right, already, far, really, why, together, probably, yet, quite, Not, How, 


ADP:  o

88312.9040357


Tags of Observation Sequence
['DET', 'NOUN', 'VERB', 'PRT', 'NOUN', 'ADP', 'NOUN']


Top 50 words for Sequence Tags
DET:  the, a, his, The, this, an, which, her, that, their, no, what, my, its, some, any, these, A, This, our, each, your, those, both, another, What, His, every, These, That, No, Her, An, My, whose, Some, Each, Another, Our, whatever, Both, Their, Your, either, Its, Those, Any, Every, Which, neither, 


NOUN:  Af, time, man, way, years, men, eyes, people, day, year, Mr., something, hand, one, room, head, door, water, place, state, number, life, face, house, work, home, system, States, night, part, course, hands, nothing, thing, world, side, information, things, end, case, line, anything, point, use, school, Mrs., business, group, voice, moment, 


VERB:  was, is, had, be, were, have, are, would, been, said, could, will, can, has, do, made, did, may, must, get, see, should, know, might, go, came, make, went, got, didn't, thought, used, come, found, looked, 

ADV:  not, when, so, only, then, now, more, back, as, where, even, just, too, also, here, there, still, very, again, never, When, away, how, well, about, always, most, much, Then, once, Now, almost, around, later, enough, ever, however, first, rather, right, already, far, really, why, together, probably, yet, quite, Not, How, 


DET:  the, a, his, The, this, an, which, her, that, their, no, what, my, its, some, any, these, A, This, our, each, your, those, both, another, What, His, every, These, That, No, Her, An, My, whose, Some, Each, Another, Our, whatever, Both, Their, Your, either, Its, Those, Any, Every, Which, neither, 


NOUN:  Af, time, man, way, years, men, eyes, people, day, year, Mr., something, hand, one, room, head, door, water, place, state, number, life, face, house, work, home, system, States, night, part, course, hands, nothing, thing, world, side, information, things, end, case, line, anything, point, use, school, Mrs., business, group, voice, moment, 


ADP:  of, in,

...

ADP:  of, in, to, for, with, on, that, as, at, by, from, into, In, if, than, about, like, through, over, before, after, If, between, under, because, against, For, so, while, without, until, out, toward, upon, As, At, since, around, during, though, within, across, On, behind, down, per, among, With, whether, Of, 


VERB:  was, is, had, be, were, have, are, would, been, said, could, will, can, has, do, made, did, may, must, get, see, should, know, might, go, came, make, went, got, didn't, thought, used, come, found, looked, take, being, don't, knew, going, say, think, felt, took, saw, turned, told, put, asked, seemed, 


NUM:  one, two, 1, 2, three, 3, One, four, five, 4, 1960, million, six, hundred, ten, 5, 1959, 1961, 10, 1958, 30, 6, 15, thousand, Two, 8, nine, seven, 100, thirty, twenty, eight, 12, 20, 7, fifty, 11, Three, eleven, 1957, twelve, 1962, billion, 25, fifteen, 31, 16, 9, 50, 0, 


.:  ,, ``, '', ;, --, ), (, :, ', ], [, 


CONJ:  and, or, but, But, And, either, nor, &, Or

8.76858631466e+25


Tags of Observation Sequence
['PRON', 'VERB', 'PRT', 'NOUN', 'ADP', 'NOUN', 'NOUN', '.', 'CONJ', 'VERB', 'ADV', 'ADP', '.', 'NOUN', 'NOUN', 'NOUN', '.', 'ADP', 'ADP', 'ADJ', 'NOUN', 'NOUN']


Top 50 words for Sequence Tags
PRON:  he, it, I, He, you, him, she, they, It, them, her, we, me, She, who, that, They, You, We, himself, us, itself, herself, themselves, myself, yourself, Who, whom, his, 'em, mine, ourselves, yours, That, hers, Him, theirs, Me, Ah, ours, Whoever, yourselves, em, ya, ye, Thou, whoever, Y'all, whatsoever, thee, 


VERB:  was, is, had, be, were, have, are, would, been, said, could, will, can, has, do, made, did, may, must, get, see, should, know, might, go, came, make, went, got, didn't, thought, used, come, found, looked, take, being, don't, knew, going, say, think, felt, took, saw, turned, told, put, asked, seemed, 


PRT:  to, all, up, out, there, There, down, on, in, off, I'm, over, I'll, such, It's, I've, I'd, half, Oh, All, Well, To, That's,

ADJ:  other, such, first, little, same, new, more, own, many, long, old, small, good, few, great, last, much, possible, big, high, different, large, only, several, important, certain, young, social, next, second, white, general, least, local, open, New, most, whole, right, less, necessary, available, dark, American, sure, various, major, low, clear, dead, 


CONJ:  and, or, but, But, And, either, nor, &, Or, neither, yet, Nor, Yet, an, plus, an', Neither, and/or, Either, An, though, An', Plus, 'n', minus, 












Observation Sequence
In 1959-60 , vehicles averaged an operating cost of $.027 per mile


Probability of the Observation Sequence
1112391736.63


Tags of Observation Sequence
['ADP', 'NUM', '.', 'NOUN', 'VERB', 'DET', 'VERB', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN']


Top 50 words for Sequence Tags
ADP:  of, in, to, for, with, on, that, as, at, by, from, into, In, if, than, about, like, through, over, before, after, If, between, under, because, against, For, so, while, witho

DET:  the, a, his, The, this, an, which, her, that, their, no, what, my, its, some, any, these, A, This, our, each, your, those, both, another, What, His, every, These, That, No, Her, An, My, whose, Some, Each, Another, Our, whatever, Both, Their, Your, either, Its, Those, Any, Every, Which, neither, 


ADJ:  other, such, first, little, same, new, more, own, many, long, old, small, good, few, great, last, much, possible, big, high, different, large, only, several, important, certain, young, social, next, second, white, general, least, local, open, New, most, whole, right, less, necessary, available, dark, American, sure, various, major, low, clear, dead, 


VERB:  was, is, had, be, were, have, are, would, been, said, could, will, can, has, do, made, did, may, must, get, see, should, know, might, go, came, make, went, got, didn't, thought, used, come, found, looked, take, being, don't, knew, going, say, think, felt, took, saw, turned, told, put, asked, seemed, 












Observation S



Observation Sequence
Requests are made by the motor pool along with any necessary cooperation from the agencies to which assignments of cars will be made


Probability of the Observation Sequence
1.24761511249e+42


Tags of Observation Sequence
['NOUN', 'VERB', 'VERB', 'ADP', 'DET', 'NOUN', 'NOUN', 'ADP', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'VERB', 'VERB', 'VERB']


Top 50 words for Sequence Tags
NOUN:  Af, time, man, way, years, men, eyes, people, day, year, Mr., something, hand, one, room, head, door, water, place, state, number, life, face, house, work, home, system, States, night, part, course, hands, nothing, thing, world, side, information, things, end, case, line, anything, point, use, school, Mrs., business, group, voice, moment, 


VERB:  was, is, had, be, were, have, are, would, been, said, could, will, can, has, do, made, did, may, must, get, see, should, know, might, go, came, make, went, got, didn't, thought, used, come